In [1]:
import random
import os
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from os import path
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


d = os.getcwd()

text_pre = open('Abstract_all.txt', encoding = "utf8").read()


doc_l = str.split(text_pre, sep = 'SEP')
#doc_l.pop()[0]

doc_complete = doc_l

doc_out = []
for l in doc_l:
    
    cleantextprep = str(l)
    
    # Regex cleaning
    expression = "[^a-zA-Z ]" # keep only letters, numbers and whitespace
    cleantextCAP = re.sub(expression, ' ', cleantextprep) # apply regex
    cleantextCAP = re.sub('\s+', ' ', cleantextCAP) # apply regex
    cleantext = cleantextCAP.lower() # lower case 
    bound = ''.join(cleantext)
    doc_out.append(bound)

# investigate doc_out list[]
print(len(doc_out))
print(doc_out[169])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


171
 abstract electricity load forecasts are an integral part of many decision making processes in the electricity market however most literature on electricity load forecasting concentrates on deterministic forecasts neglecting possibly important information about uncertainty a more complete picture of future demand can be obtained by using distributional forecasts allowing for a more efficient decision making a predictive density can be fully characterized by tail measures such as quantiles and expectiles furthermore interest often lies in the accurate estimation of tail events rather than in the mean or median we propose a new methodology to obtain probabilistic forecasts of electricity load that is based on functional data analysis of generalized quantile curves the core of the methodology is dimension reduction based on functional principal components of tail curves with dependence structure the approach has several advantages such as flexible inclusion of explanatory variables in

In [2]:
doc_complete = doc_out

stop = set(stopwords.words('english'))
stop = stop.union({'result','keywords','study','using','paper','abstract','f','x','e','result','topic','proposed','one'})
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]    

print(doc_clean[0])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['estimating', 'spot', 'covariance', 'important', 'issue', 'especially', 'theincreasing', 'availability', 'high', 'frequency', 'nancial', 'data', 'estimationof', 'spot', 'covariance', 'kernel', 'method', 'high', 'frequency', 'data', 'particular', 'consider', 'rst', 'kernel', 'weighted', 'version', 'realized', 'covariance', 'estimatorfor', 'price', 'process', 'governed', 'continuous', 'multivariate', 'semimartingale', 'next', 'extend', 'threshold', 'kernel', 'estimator', 'spot', 'covarianceswhen', 'underlying', 'price', 'process', 'discontinuous', 'multivariate', 'semimartingalewith', 'nite', 'activity', 'jump', 'derive', 'asymptotic', 'distribution', 'theestimators', 'xed', 'shrinking', 'bandwidth', 'estimator', 'settingwith', 'jump', 'rate', 'convergence', 'estimator', 'di', 'usionprocesses', 'without', 'jump', 'simulation', 'examines', 'nite', 'sample', 'propertiesof', 'estimator', 'addition', 'application', 'estimator', 'thecontext', 'covariance', 'forecasting', 'discover', 'forecas

In [15]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Human readable format of corpus (term-frequency)
word_freq = [[(dictionary[id], freq) for id, freq in cp] for cp in doc_term_matrix[:1]]
nl = len(word_freq[0])
wlst = []
flst = []
for i in range(nl):
    key_word = word_freq[0][i][0]
    freq_word = word_freq[0][i][1]
    wlst.append(key_word)
    flst.append(freq_word)
wdfq = np.vstack((wlst,flst)).T
df_wf = pd.DataFrame(wdfq)
df_wf.columns =['Key word', 'Frequency']
print(df_wf)

              Key word Frequency
0             activity         1
1             addition         1
2          application         1
3           asymptotic         1
4         availability         1
5            bandwidth         1
6            benchmark         1
7             consider         1
8           continuous         1
9          convergence         1
10          covariance         5
11     covarianceswhen         1
12                data         3
13              derive         1
14                  di         1
15       discontinuous         1
16            discover         1
17        distribution         1
18          especially         1
19          estimating         1
20          estimation         1
21        estimationof         1
22           estimator         6
23        estimatorfor         1
24            examines         1
25              extend         1
26         forecasting         3
27           frequency         3
28            governed         1
29        

In [5]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, 
               num_topics=5, 
               id2word = dictionary, 
               passes=50, 
               random_state = 3154,
               alpha='auto',
               per_word_topics=True)

#the keywords for each topic and the weightage(importance) of each keyword
from pprint import pprint
pprint(ldamodel.print_topics(num_topics=3, num_words=5))


[(2,
  '0.020*"model" + 0.011*"method" + 0.011*"data" + 0.011*"variable" + '
  '0.007*"estimator"'),
 (3,
  '0.016*"model" + 0.011*"risk" + 0.008*"distribution" + 0.008*"time" + '
  '0.007*"data"'),
 (4,
  '0.016*"market" + 0.015*"risk" + 0.009*"index" + 0.007*"network" + '
  '0.007*"financial"')]


In [6]:
K=5
topicWordProbMat=ldamodel.print_topics(K)
pprint(topicWordProbMat)
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix)) 

# Compute Coherence Score
from gensim.models import CoherenceModel
coherence_ldamodel = CoherenceModel(model=ldamodel, 
                                    texts=doc_clean, 
                                    dictionary=dictionary, 
                                    coherence='c_v')

coherence_lda = coherence_ldamodel.get_coherence()
print('\nCoherence Score: ', coherence_lda)

[(0,
  '0.018*"model" + 0.009*"network" + 0.008*"learning" + 0.007*"data" + '
  '0.007*"risk" + 0.006*"inflation" + 0.006*"financial" + 0.006*"deep" + '
  '0.005*"factor" + 0.005*"research"'),
 (1,
  '0.017*"model" + 0.014*"risk" + 0.012*"market" + 0.010*"volatility" + '
  '0.007*"time" + 0.006*"portfolio" + 0.006*"cryptocurrencies" + 0.006*"data" '
  '+ 0.005*"tail" + 0.005*"price"'),
 (2,
  '0.020*"model" + 0.011*"method" + 0.011*"data" + 0.011*"variable" + '
  '0.007*"estimator" + 0.006*"parameter" + 0.005*"nonparametric" + '
  '0.005*"adaptive" + 0.005*"change" + 0.005*"test"'),
 (3,
  '0.016*"model" + 0.011*"risk" + 0.008*"distribution" + 0.008*"time" + '
  '0.007*"data" + 0.006*"method" + 0.005*"factor" + 0.005*"gaussian" + '
  '0.005*"asset" + 0.005*"high"'),
 (4,
  '0.016*"market" + 0.015*"risk" + 0.009*"index" + 0.007*"network" + '
  '0.007*"financial" + 0.007*"model" + 0.006*"based" + 0.005*"currency" + '
  '0.005*"tail" + 0.005*"crix"')]

Perplexity:  -7.63496186687069

Cohe

In [7]:
# VISUALIZATION lda OUTPUT

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\ASUS\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.103961 -0.001025       1        1  26.923424
3     -0.022815  0.029831       2        1  22.271467
1      0.069038  0.035647       3        1  18.462847
0     -0.043367 -0.134888       4        1  16.309784
2     -0.106817  0.070434       5        1  16.032482, topic_info=     Category        Freq              Term       Total  loglift  logprob
128   Default   57.000000          variable   57.000000  30.0000  30.0000
460   Default   60.000000        volatility   60.000000  29.0000  29.0000
37    Default  246.000000             model  246.000000  28.0000  28.0000
311   Default  139.000000            market  139.000000  27.0000  27.0000
547   Default   35.000000          learning   35.000000  26.0000  26.0000
935   Default   17.000000              deep   17.000000  25.0000  25.0000
160   Default   19.000000         inflation   19.000000  24.0000  24.0000
94    Default   56.000000         financial   56.000000  23.0000  23.0000
156   Default   23.000000          gaussian   23.000000  22.0000  22.0000
22    Default   30.000000         estimator   30.000000  21.0000  21.0000
838   Default   29.000000              crix   29.000000  20.0000  20.0000
660   Default   54.000000             index   54.000000  19.0000  19.0000
528   Default   13.000000            spline   13.000000  18.0000  18.0000
573   Default   29.000000           trading   29.000000  17.0000  17.0000
36    Default   89.000000            method   89.000000  16.0000  16.0000
1436  Default   22.000000       correlation   22.000000  15.0000  15.0000
1487  Default   18.000000            random   18.000000  14.0000  14.0000
652   Default   35.000000  cryptocurrencies   35.000000  13.0000  13.0000
1280  Default   28.000000          adaptive   28.000000  12.0000  12.0000
1157  Default   22.000000           default   22.000000  11.0000  11.0000
100   Default   13.000000      instrumental   13.000000  10.0000  10.0000
103   Default   77.000000           network   77.000000   9.0000   9.0000
12    Default  110.000000              data  110.000000   8.0000   8.0000
701   Default   29.000000          currency   29.000000   7.0000   7.0000
60    Default   13.000000         threshold   13.000000   6.0000   6.0000
151   Default   11.000000       expectation   11.000000   5.0000   5.0000
653   Default   35.000000    cryptocurrency   35.000000   4.0000   4.0000
343   Default   21.000000             break   21.000000   3.0000   3.0000
316   Default   46.000000         portfolio   46.000000   2.0000   2.0000
1689  Default   16.000000               frm   16.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
22     Topic5   17.763990         estimator   30.571356   1.2877  -5.0327
487    Topic5    8.607686             monte   13.396511   1.3882  -5.7572
464    Topic5    8.607684             carlo   13.396511   1.3882  -5.7572
1824   Topic5    4.039379            proper    5.543184   1.5141  -6.5137
128    Topic5   29.080742          variable   57.687664   1.1456  -4.5398
1280   Topic5   14.780606          adaptive   28.908592   1.1597  -5.2165
36     Topic5   31.262794            method   89.170486   0.7824  -4.4674
802    Topic5   12.288456            linear   24.842690   1.1267  -5.4012
804    Topic5   14.793045     nonparametric   32.123035   1.0551  -5.2157
1133   Topic5    7.146863    regularization   11.918319   1.3192  -5.9432
37     Topic5   53.857555             model  246.606155   0.3091  -3.9235
12     Topic5   31.106041              data  110.720695   0.5609  -4.4724
440    Topic5   15.387421         parameter   39.078804   0.8985  -5.1763
1404   Topic5    7.146668          interval   12.703627   1.2553  -5.9432
712    Topic5    5.592826     specification    8.775785   1.3800  -6.1883
318    Topic5    4.814288            premia    7.025647   1.4526  -6.3382
346    Topic5   14.007229  

In [9]:
from pprint import pprint
from gensim.models import CoherenceModel
k=3
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel3 = Lda(doc_term_matrix, 
               num_topics=k, 
               id2word = dictionary, 
               passes=50, 
               random_state = 3154,
               alpha='auto',
               per_word_topics=True)

coherence_ldamodel3 = CoherenceModel(model=ldamodel3, 
                                    texts=doc_clean, 
                                    dictionary=dictionary, 
                                    coherence='c_v')

topicWordProbMat=ldamodel3.print_topics(k)
pprint(topicWordProbMat)
print('\nPerplexity of k=3 : ', ldamodel3.log_perplexity(doc_term_matrix)) 
coherence_lda = coherence_ldamodel3.get_coherence()
print('\nCoherence Score of k=3 : ', coherence_lda)

[(0,
  '0.014*"model" + 0.011*"network" + 0.010*"risk" + 0.006*"financial" + '
  '0.006*"data" + 0.006*"factor" + 0.005*"learning" + 0.004*"systemic" + '
  '0.004*"result" + 0.004*"performance"'),
 (1,
  '0.016*"market" + 0.016*"risk" + 0.014*"model" + 0.008*"volatility" + '
  '0.007*"time" + 0.007*"tail" + 0.006*"portfolio" + 0.006*"index" + '
  '0.006*"asset" + 0.006*"based"'),
 (2,
  '0.017*"model" + 0.010*"data" + 0.010*"method" + 0.008*"variable" + '
  '0.006*"time" + 0.005*"change" + 0.005*"high" + 0.005*"estimation" + '
  '0.005*"nonparametric" + 0.005*"parameter"')]

Perplexity of k=3 :  -7.606212531637758

Coherence Score of k=3 :  0.3492760442979739


In [10]:
vis = pyLDAvis.gensim.prepare(ldamodel3, doc_term_matrix, dictionary)
vis

C:\Users\ASUS\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.093466 -0.045595       1        1  41.040779
0     -0.086554 -0.057466       2        1  30.735397
2     -0.006912  0.103062       3        1  28.223824, topic_info=     Category        Freq              Term       Total  loglift  logprob
311   Default  140.000000            market  140.000000  30.0000  30.0000
103   Default   76.000000           network   76.000000  29.0000  29.0000
460   Default   62.000000        volatility   62.000000  28.0000  28.0000
316   Default   47.000000         portfolio   47.000000  27.0000  27.0000
838   Default   29.000000              crix   29.000000  26.0000  26.0000
346   Default   40.000000            change   40.000000  25.0000  25.0000
343   Default   21.000000             break   21.000000  24.0000  24.0000
128   Default   57.000000          variable   57.000000  23.0000  23.0000
160   Default   20.000000         inflation   20.000000  22.0000  22.0000
122   Default   63.000000              tail   63.000000  21.0000  21.0000
653   Default   36.000000    cryptocurrency   36.000000  20.0000  20.0000
547   Default   35.000000          learning   35.000000  19.0000  19.0000
935   Default   17.000000              deep   17.000000  18.0000  18.0000
652   Default   36.000000  cryptocurrencies   36.000000  17.0000  17.0000
1707  Default   28.000000          systemic   28.000000  16.0000  16.0000
36    Default   89.000000            method   89.000000  15.0000  15.0000
573   Default   30.000000           trading   30.000000  14.0000  14.0000
1111  Default   50.000000             asset   50.000000  13.0000  13.0000
94    Default   55.000000         financial   55.000000  12.0000  12.0000
411   Default   27.000000           bitcoin   27.000000  11.0000  11.0000
804   Default   31.000000     nonparametric   31.000000  10.0000  10.0000
528   Default   13.000000            spline   13.000000   9.0000   9.0000
1689  Default   16.000000               frm   16.000000   8.0000   8.0000
194   Default   15.000000              bank   15.000000   7.0000   7.0000
114   Default  180.000000              risk  180.000000   6.0000   6.0000
358   Default   26.000000       dimensional   26.000000   5.0000   5.0000
660   Default   54.000000             index   54.000000   4.0000   4.0000
1527  Default   17.000000        structural   17.000000   3.0000   3.0000
22    Default   31.000000         estimator   31.000000   2.0000   2.0000
12    Default  110.000000              data  110.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
9      Topic3   11.897146       convergence   15.816755   0.9802  -5.9991
2193   Topic3    8.057451            growth   10.283623   1.0210  -6.3888
804    Topic3   22.168020     nonparametric   31.796124   0.9043  -5.3767
128    Topic3   37.065498          variable   57.382412   0.8280  -4.8627
346    Topic3   26.154095            change   40.976288   0.8160  -5.2114
22     Topic3   20.373526         estimator   31.033703   0.8442  -5.4611
33     Topic3    9.639726            kernel   12.734346   0.9866  -6.2095
36     Topic3   45.735821            method   89.183044   0.5972  -4.6525
376    Topic3   12.551577             point   18.278963   0.8891  -5.9455
1280   Topic3   17.400991          adaptive   28.786015   0.7616  -5.6188
20     Topic3   24.492802        estimation   44.831593   0.6605  -5.2770
12     Topic3   46.940983              data  110.999619   0.4044  -4.6265
440    Topic3   21.608831         parameter   39.415436   0.6639  -5.4023
29     Topic3   25.418404              high   49.985542   0.5887  -5.2399
37     Topic3   79.894997             model  248.848633   0.1289  -4.0947
379    Topic3   18.564323         procedure   34.275429   0.6518  -5.5541
329    Topic3   21.110468              test   43.197659   0.5490  -5.4256
802    Topic3   14.536515            linear   24.672840   0.7

In [11]:
from pprint import pprint
from gensim.models import CoherenceModel
k=10
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_10 = Lda(doc_term_matrix, 
               num_topics=k, 
               id2word = dictionary, 
               passes=50, 
               random_state = 3154,
               alpha='auto',
               per_word_topics=True)

coherence_ldamodel_10 = CoherenceModel(model=ldamodel_10, 
                                    texts=doc_clean, 
                                    dictionary=dictionary, 
                                    coherence='c_v')

topicWordProbMat=ldamodel_10.print_topics(k)
pprint(topicWordProbMat)
print('\nPerplexity of k=10 : ', ldamodel_10.log_perplexity(doc_term_matrix)) 
coherence_lda = coherence_ldamodel_10.get_coherence()
print('\nCoherence Score of k=10 : ', coherence_lda)

[(0,
  '0.011*"network" + 0.009*"model" + 0.008*"deep" + 0.008*"learning" + '
  '0.006*"data" + 0.006*"test" + 0.006*"sample" + 0.006*"factor" + '
  '0.005*"forecasting" + 0.005*"method"'),
 (1,
  '0.022*"model" + 0.017*"market" + 0.014*"volatility" + 0.014*"risk" + '
  '0.008*"cryptocurrencies" + 0.008*"data" + 0.007*"time" + 0.007*"based" + '
  '0.007*"dynamic" + 0.006*"trading"'),
 (2,
  '0.015*"method" + 0.013*"model" + 0.011*"data" + 0.009*"variable" + '
  '0.007*"estimator" + 0.007*"spline" + 0.007*"convergence" + 0.006*"adaptive" '
  '+ 0.006*"high" + 0.006*"risk"'),
 (3,
  '0.016*"model" + 0.011*"time" + 0.009*"distribution" + 0.009*"asset" + '
  '0.009*"break" + 0.008*"gaussian" + 0.008*"risk" + 0.007*"high" + '
  '0.006*"procedure" + 0.006*"method"'),
 (4,
  '0.022*"market" + 0.016*"risk" + 0.014*"index" + 0.009*"crix" + 0.008*"frm" '
  '+ 0.008*"measure" + 0.006*"cryptocurrency" + 0.006*"sentiment" + '
  '0.006*"option" + 0.006*"financial"'),
 (5,
  '0.019*"default" + 0.014*

In [16]:
vis = pyLDAvis.gensim.prepare(ldamodel_10, doc_term_matrix, dictionary)
vis

C:\Users\ASUS\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.080853 -0.032459       1        1  16.569826
6      0.034634 -0.090881       2        1  12.900117
1      0.071451  0.030612       3        1  12.792355
3      0.062851 -0.072002       4        1  12.072145
4      0.062866  0.127419       5        1  11.399048
9      0.048293  0.099411       6        1  11.207970
0     -0.129887 -0.062499       7        1   7.954253
2      0.011964 -0.099629       8        1   7.298084
8     -0.102593  0.063967       9        1   6.639887
5     -0.140433  0.036062      10        1   1.166320, topic_info=     Category        Freq              Term       Total  loglift  logprob
1157  Default   17.000000           default   17.000000  30.0000  30.0000
103   Default   80.000000           network   80.000000  29.0000  29.0000
556   Default   33.000000       performance   33.000000  28.0000  28.0000
122   Default   64.000000              tail   64.000000  27.0000  27.0000
311   Default  139.000000            market  139.000000  26.0000  26.0000
128   Default   59.000000          variable   59.000000  25.0000  25.0000
36    Default   91.000000            method   91.000000  24.0000  24.0000
660   Default   54.000000             index   54.000000  23.0000  23.0000
160   Default   21.000000         inflation   21.000000  22.0000  22.0000
460   Default   61.000000        volatility   61.000000  21.0000  21.0000
343   Default   22.000000             break   22.000000  20.0000  20.0000
838   Default   29.000000              crix   29.000000  19.0000  19.0000
701   Default   28.000000          currency   28.000000  18.0000  18.0000
156   Default   24.000000          gaussian   24.000000  17.0000  17.0000
1689  Default   16.000000               frm   16.000000  16.0000  16.0000
652   Default   36.000000  cryptocurrencies   36.000000  15.0000  15.0000
194   Default   14.000000              bank   14.000000  14.0000  14.0000
935   Default   17.000000              deep   17.000000  13.0000  13.0000
37    Default  257.000000             model  257.000000  12.0000  12.0000
547   Default   35.000000          learning   35.000000  11.0000  11.0000
659   Default   19.000000            impact   19.000000  10.0000  10.0000
1707  Default   30.000000          systemic   30.000000   9.0000   9.0000
316   Default   47.000000         portfolio   47.000000   8.0000   8.0000
32    Default   27.000000              jump   27.000000   7.0000   7.0000
324   Default   11.000000               run   11.000000   6.0000   6.0000
310   Default   18.000000              long   18.000000   5.0000   5.0000
109   Default   40.000000          quantile   40.000000   4.0000   4.0000
1111  Default   49.000000             asset   49.000000   3.0000   3.0000
239   Default   20.000000             short   20.000000   2.0000   2.0000
142   Default   18.000000            copula   18.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
2577  Topic10    0.368691            damage    1.103227   3.3553  -6.2869
2608  Topic10    0.368691              zone    1.103227   3.3553  -6.2869
2602  Topic10    0.368691          shanghai    1.103227   3.3553  -6.2869
2604  Topic10    0.368691          suffered    1.103227   3.3553  -6.2869
3642  Topic10    0.368668        extraction    1.103210   3.3552  -6.2869
3640  Topic10    0.368668              debt    1.103210   3.3552  -6.2869
3650  Topic10    0.368668         stabilize    1.103210   3.3552  -6.2869
2603  Topic10    0.703905             skill    2.266740   3.2819  -5.6402
2607  Topic10    0.703905            winter    2.276634   3.2775  -5.6402
1157  Topic10    3.690352           default   17.222843   2.9108  -3.9833
671   Topic10    1.039113            online    5.860600   2.7214  -5.2507
1363  Topic10    1.374321             human    9.579694   2.5096  -4.9711
556   Topic10    2.715149       performance   33.2964